### ver01 : 네이버페이 선착순구매 매크로 참고

##### "장인,더" 매크로 만들기!!

###### ~ 장인약과를 양손에 쥐고 먹는 그날까지 ~

##### 1) import packages

In [7]:
import os
import threading
import platform
import subprocess
import time
import pyautogui
from pynput.mouse import Button, Controller
# import cv2
import datetime
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import ElementClickInterceptedException, imeoutException, NoSuchElementException,  NoSuchWindowException, WebDriverException


##### 2) data

In [19]:
### 
def copy_input(xpath, input):
    pyperclip.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(0.1)

In [8]:
if os.name == 'nt':
    from ctypes import windll

    is_retina = False
    if platform.system() == "Darwin":
        is_retina = subprocess.call("system_profiler SPDisplaysDataType | grep 'retina'", shell=True)
else:
    is_retina = True

mouse = Controller()

In [9]:
### N개로 수행할 수 있도록 클래스 생성
class Browser:
    def __init__(self, user_id, password, pay_number):
        self.timeout = 2
        self.user_id = user_id
        self.password = password
        self.pay_number = pay_number

        self.base_width = 1920
        self.base_height = 1080

        # 모니터 해상도 가져오기
        self.scr_width, self.scr_height = pyautogui.size()
        print('width={0}, height={1}'.format(self.scr_width, self.scr_height))

        self.w_rat = self.base_width / self.scr_width
        self.h_rat = self.base_height / self.scr_height

        if os.name == 'nt':
            user32 = windll.user32
            user32.SetProcessDPIAware()

In [10]:
 def popup(self):
        chrome_options = Options()
        #chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('window-size=1920x1080')  # 가져올 크기를 결정

        self.driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

In [11]:
### 구매시 옵션 선택
def setOption(self, url, combo=True, option='', another=True):
        self.url = url
        self.combo = combo
        self.option = option
        self.another = another

In [12]:
### 로그인 수행
def login(self):
        # 장인약과 홈페이지 Login
        self.driver.get('https://janginthe.com/member/login.html')
        self.driver.execute_script("document.getElementsByName('id')[0].value=\'" + self.user_id + "\'") # 자동로그인방지를 우회하기 위해 스크립트로 id와 PW 변경
        self.driver.execute_script("document.getElementsByName('pw')[0].value=\'" + self.password + "\'")
        WebDriverWait(self.driver, self.timeout).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="frmNIDLogin"]/fieldset/input'))).click()
        # self.driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

        print('{} 로그인'.format(self.user_id))
        time.sleep(1)

In [16]:
### 상품의 재고 여부 조회 (재고가 존재할때까지 반복 수행)
def checkStock(self):
        print('{} 상품 재고 조회'.format(self.user_id))
        fiedset = WebDriverWait(self.driver, self.timeout).until(EC.element_to_be_clickable((By.CLASS_NAME, 'prd_num')))

        while True:
            self.driver.refresh()
            try:
                not_good = self.driver.find_element_by_class_name('not_goods')
            except NoSuchElementException:
                not_good = ''
                break

        if self.combo:
            try:
                self.option_combo = WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, '_combination_option')))
                all_options = self.option_combo.find_elements_by_tag_name("option")

                try:
                    product_list = WebDriverWait(self.driver, self.timeout).until(EC.element_to_be_clickable((By.CLASS_NAME, 'opt_price')))
                    index = next(idx for idx, option in enumerate(all_options) if
                                 option.get_attribute("text") not in ["사이즈", "필수옵션"] and option.get_attribute(
                                     "title") == self.option)

                    # 입력한 옵션값이 있을 경우 품절여부 판단
                    if index > 0:
                        sold_out = all_options[1].get_attribute("text").find("품절") > -1

                        # 입력한 옵션은 품절이지만 다른 상품구매도 원한다면 변경하여 진행
                        if sold_out and self.another:
                            try:
                                index = next(idx for idx, option in enumerate(all_options) if
                                             option.get_attribute("text") not in ["사이즈", "필수옵션"] and option.get_attribute(
                                                 "text").find("품절") == -1)
                            except StopIteration:
                                print('전상품 품절')
                                time.sleep(600)
                        elif sold_out:
                            print('{} 품절 - 재조회'.format(self.option))
                            self.driver.get(self.url)
                            thread = threading.Thread(target=self.checkStock)
                            thread.start()

                except StopIteration:
                    print('입력값에 해당하는 옵션 없음'.format(self.option))
                    if self.another:
                        try:
                            index = next(idx for idx, option in enumerate(all_options) if
                                         option.get_attribute("text") not in ["사이즈", "필수옵션"] and option.get_attribute(
                                             "text").find("품절") == -1)
                        except StopIteration:
                            print('전상품 품절')
                            time.sleep(600)
                    else:
                        print('더이상 진행하지 않음')
                        time.sleep(600)
            except TimeoutException:
                print('서버폭주로 비활성화')
                self.driver.get(self.url)
                thread = threading.Thread(target=self.checkStock)
                thread.start()

            if self.another:
                input_option = all_options[index].get_attribute("title")
                self.option_combo.send_keys(input_option)
            else:
                try:
                    self.option_combo.send_keys(self.option)
                except AttributeError:
                    print('Combobox Error')
                    self.driver.get(self.url)
                    thread = threading.Thread(target=self.checkStock)
                    thread.start()

        self.checkout()

In [18]:
def selectItem(self, option_combo):
        if self.combo:
            try:
                purchase_list = self.driver.find_elements_by_class_name('_purchase_unit')

                if purchase_list:
                    for item in purchase_list:
                        item_text = item.find_elements_by_tag_name('em')[0].text
                        if self.option != item_text:
                            btn_del = item.find_element_by_partial_link_text("삭제")
                            btn_del.click()

                    purchase_list = self.driver.find_elements_by_class_name('_purchase_unit')

                    if purchase_list == []:
                        option_combo.send_keys(self.option)
                else:
                    option_combo.send_keys(self.option)
            except NoSuchElementException:
                option_combo.send_keys(self.option)

    def order(self):
        btn_pay = WebDriverWait(self.driver, self.timeout).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'btn_payment')))

        # storing the current window handle to get back to dashbord
        main_page = self.driver.current_window_handle

        btn_pay.click()

        print('{} 구매완료'.format(self.user_id))
        # 10분간 대기 이후 Thread는 종료
        time.sleep(600)

    def startBrowser(self):
        self.popup()

        self.login()

        # Url로 이동
        self.driver.get(self.url)
        print('{} Url로 이동'.format(self.user_id))

        self.checkStock()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 22)

In [ ]:
browser1 = Browser(아이디, 패스워드, 페이번호)
browser1.setOption('https://smartstore.naver.com/mfbshop/products/4819214174', True, 'Small (소)', False)
browser_thread1 = threading.Thread(target=browser1.startBrowser)
browser_thread1.start()